In [1]:
import sys
import untangle
import socnet as sn
import networkx as nx
import seaborn as sns
import numpy as np
from collections import Counter
from scipy.stats.stats import pearsonr
from scipy.optimize import minimize
from scipy import stats

In [2]:
tags = untangle.parse("./data/Tags.xml")
postHist = untangle.parse("./data/PostHistory.xml")
posts = untangle.parse("./data/Posts.xml")
users = untangle.parse("./data/Users.xml")
votes = untangle.parse("./data/Votes.xml")

In [3]:
sn.node_size = 10
sn.node_color = (255, 255, 255)

sn.edge_width = 0.5
sn.edge_color = (192, 192, 192)

sn.node_label_position = 'top center'

In [ ]:
def normalizeList(lista):

In [5]:
#{userid : [postid1, postid2]} - userposts
#{postid : parentpost} - parent
#{postid : user} - postuser
#{userid : displayname} - username
#{userid : reputation} - userrep
#{normalizedid : userid} normalizedid = {}
#[(userid, userid)] - edges

userposts = {}
parent = {}
postuser = {}
username = {}
userrep = {}
normalizedid = {}
edges = []

fakes = {"user306": "9001", "user2508039": "9002", "Shireen": "2047"}
fakerep = [0,0,66]
#There are 3 users that appears names on posts references, but are not mentioned anywhere else  
#we added them to a unique dictionary so they would not colapse the entire graph
#Shireen is actually a working user entry, but it seens the this user changed ts nickname so maybe thats why it also shows its
#old name in the post. Another postt she appears does not have the OwnerDisplayName field

counter = 0
for i in users.users.row:
    normalizedid[counter] = i["Id"]
    userposts[i['Id']] = []
    username[i['Id']] = str(i['DisplayName'].encode("utf-8", 'xmlcharrefreplace')).strip("b'").strip("'").strip("\"")
    userrep[i['Id']] = int(i["Reputation"])
    counter +=1
freezeCounter = counter
for i in fakes.items():
    userposts[i[1]] = []
    username[i[1]] = i[0]
    username[i[1]] = i[0]
    userrep[i[1]] = fakerep[counter - freezeCounter]
    normalizedid[counter] = i[1]
    counter +=1 

for i in posts.posts.row:
    
    
    current_parent = None
    if i["PostTypeId"] == "2":
        current_parent = i["ParentId"]
    parent[i['Id']] = current_parent
    
    #Gerenciate (engrish pls) cases when there is a wierd name instead of id, as represented in <Dict>fakes
    if(i["OwnerUserId"]) == None:
        postuser[i["Id"]] = fakes[i["OwnerDisplayName"]]
        userposts[fakes[i["OwnerDisplayName"]]].append(i["Id"])
        
    else:
        postuser[i["Id"]] = i["OwnerUserId"]
        userposts[i["OwnerUserId"]].append(i["Id"])
    


In [6]:
#write gml

file = open("quantum_V01.gml", "w")
file.write('graph [\n')
file.write('  directed 1\n')

for i in userposts.keys():
    #print(username[i])
    file.write('  node [\n')
    file.write('    id {}\n'.format(i))
    file.write('    label "' + username[i] + '"\n')
    file.write('  ]\n')
    

edges = []
for i in userposts.keys():
    for post in userposts[i]:
        question = parent[post]
        if(question != None):
            if not (i,postuser[question]) in edges:
                edges.append((i,postuser[question]))
                #print(i, " - ", postuser[question])
                file.write('  edge [\n')
                file.write('    source {}\n'.format(i))
                file.write('    target {}\n'.format(postuser[question]))
                file.write('  ]\n')
            else:
                print("Avoided crisis  -  ",i, " - ", postuser[question])
file.write(']')
file.close()

Avoided crisis  -   18  -  27
Avoided crisis  -   23  -  26
Avoided crisis  -   23  -  26
Avoided crisis  -   23  -  55
Avoided crisis  -   23  -  253
Avoided crisis  -   23  -  26
Avoided crisis  -   23  -  2645
Avoided crisis  -   26  -  261
Avoided crisis  -   26  -  26
Avoided crisis  -   26  -  91
Avoided crisis  -   26  -  609
Avoided crisis  -   26  -  26
Avoided crisis  -   26  -  2559
Avoided crisis  -   26  -  2559
Avoided crisis  -   26  -  2559
Avoided crisis  -   26  -  2559
Avoided crisis  -   26  -  26
Avoided crisis  -   26  -  2529
Avoided crisis  -   26  -  3035
Avoided crisis  -   54  -  1658
Avoided crisis  -   54  -  409
Avoided crisis  -   54  -  2371
Avoided crisis  -   55  -  261
Avoided crisis  -   55  -  104
Avoided crisis  -   55  -  580
Avoided crisis  -   55  -  1931
Avoided crisis  -   55  -  2293
Avoided crisis  -   55  -  2645
Avoided crisis  -   91  -  253
Avoided crisis  -   91  -  2645
Avoided crisis  -   104  -  91
Avoided crisis  -   119  -  2100
Av

In [45]:
nodes_raw = sn.load_graph('quantum_V01.gml', has_pos=False)
for i in range(len(edges)):
    edges[i] =  (int(edges[i][0]),int(edges[i][1]))
    
nodes = nx.edge_subgraph(nodes_raw, edges)

sn.show_graph(nodes, nlab=False)


C:\Users\Pedro\Desktop\StackERedes-master\socnet.py:59: RuntimeWarning:

divide by zero encountered in double_scalars

C:\Users\Pedro\Desktop\StackERedes-master\socnet.py:187: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\Pedro\Desktop\StackERedes-master\socnet.py:188: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\Pedro\Desktop\StackERedes-master\socnet.py:189: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\Pedro\Desktop\StackERedes-master\socnet.py:190: RuntimeWarning:

invalid value encountered in double_scalars



In [8]:
#listing how many questions were asked/answered by a certain user

(Counter(elem[0] for elem in edges))

Counter({7: 1,
         11: 1,
         13: 3,
         15: 9,
         18: 5,
         23: 20,
         24: 4,
         26: 36,
         27: 2,
         29: 1,
         32: 4,
         35: 1,
         39: 1,
         54: 13,
         55: 32,
         67: 1,
         70: 2,
         72: 2,
         82: 1,
         86: 2,
         91: 14,
         99: 2,
         104: 9,
         119: 26,
         124: 33,
         127: 2,
         138: 2,
         144: 3,
         162: 2,
         171: 1,
         176: 1,
         197: 4,
         236: 2,
         238: 4,
         253: 10,
         263: 6,
         274: 4,
         278: 22,
         299: 1,
         302: 3,
         356: 3,
         362: 2,
         391: 5,
         407: 1,
         409: 57,
         410: 3,
         419: 4,
         422: 1,
         429: 3,
         434: 11,
         483: 4,
         484: 1,
         491: 13,
         505: 1,
         506: 1,
         522: 1,
         534: 3,
         551: 4,
         563: 2,
        

In [9]:
#sorting user id by reputation

userrepSort = sorted(userrep.items(), key=lambda kv: kv[0], reverse=False)
userrepSort

[('-1', 1),
 ('1', 101),
 ('10', 101),
 ('100', 101),
 ('1000', 1),
 ('1001', 101),
 ('1002', 101),
 ('1003', 101),
 ('1004', 1),
 ('1005', 107),
 ('1006', 101),
 ('1007', 1),
 ('1008', 17),
 ('1009', 101),
 ('101', 101),
 ('1010', 101),
 ('1011', 101),
 ('1012', 1),
 ('1013', 101),
 ('1014', 1),
 ('1015', 101),
 ('1016', 1),
 ('1017', 101),
 ('1018', 101),
 ('1019', 101),
 ('102', 101),
 ('1020', 101),
 ('1021', 101),
 ('1022', 1),
 ('1023', 1),
 ('1024', 101),
 ('1025', 101),
 ('1026', 101),
 ('1027', 1),
 ('1028', 101),
 ('1029', 101),
 ('103', 1),
 ('1030', 101),
 ('1031', 101),
 ('1032', 101),
 ('1033', 101),
 ('1034', 281),
 ('1035', 1),
 ('1036', 1),
 ('1037', 1),
 ('1038', 1),
 ('1039', 1444),
 ('104', 1092),
 ('1040', 101),
 ('1041', 101),
 ('1042', 101),
 ('1043', 1),
 ('1044', 148),
 ('1045', 1),
 ('1046', 101),
 ('1047', 1),
 ('1048', 1),
 ('1049', 101),
 ('105', 101),
 ('1050', 101),
 ('1051', 101),
 ('1052', 1),
 ('1053', 101),
 ('1054', 101),
 ('1055', 1),
 ('1056', 101)

In [33]:
#betweenness for nodes with edges (values must be > 0)

bc = nx.betweenness_centrality(nodes)

In [36]:
#closeness for nodes with edges (values must be > 0)

clos = nx.closeness_centrality(nodes)
closDict = {}
        
sortDictC = sorted([(v,k) for k,v in clos.items()], reverse = True)
sortDictC

[(0.15270905870136214, 2645),
 (0.14651793789568887, 2559),
 (0.1375294042850785, 1931),
 (0.12663678038649545, 253),
 (0.12663678038649545, 55),
 (0.12561551602853982, 2293),
 (0.12461059190031153, 26),
 (0.12362161894872174, 391),
 (0.12313299594892442, 91),
 (0.12216724696108973, 1905),
 (0.11981787682722261, 1955),
 (0.11555933163409797, 1348),
 (0.11514063840353966, 138),
 (0.1149544205722431, 2371),
 (0.11328235627301048, 278),
 (0.11318439807109631, 2015),
 (0.11279810319713353, 1254),
 (0.11269083758644308, 3021),
 (0.11246443312302484, 2927),
 (0.11241443617945619, 3088),
 (0.11214953271028037, 429),
 (0.1120598848024384, 144),
 (0.11203337029410211, 90),
 (0.11165823646981318, 409),
 (0.11125945705384957, 1386),
 (0.11083784226922445, 58),
 (0.11042311661506707, 769),
 (0.11024222773221437, 1874),
 (0.10939838977222154, 11),
 (0.1093847352024922, 27),
 (0.10883156232663337, 1185),
 (0.10876246628614439, 1346),
 (0.10742292405199269, 1847),
 (0.10699938114268331, 261),
 (0.105

In [12]:
pr = nx.pagerank(nodes)
prNamed = {}
    
prNamedSorted = sorted(pr.items(), key=lambda kv: kv[1], reverse=True)
prNamedSorted

[(1346, 0.02926891349161315),
 (2645, 0.021687883787350447),
 (253, 0.02052402629635363),
 (2782, 0.018421194053108533),
 (2559, 0.0177605932845025),
 (27, 0.013815774523395533),
 (26, 0.012945196535365631),
 (2047, 0.012266005011241457),
 (2794, 0.011694333394708033),
 (55, 0.011601173650941309),
 (1905, 0.011059554430073112),
 (506, 0.011009527151459739),
 (2592, 0.011009527151459739),
 (1931, 0.010646178880198304),
 (769, 0.010123158786021038),
 (99, 0.009587213876959558),
 (1955, 0.009585841266630264),
 (11, 0.009418075054094581),
 (91, 0.009183720071682697),
 (2293, 0.009177265889328622),
 (2100, 0.008896784468144514),
 (1837, 0.00810382027317538),
 (3088, 0.007973353865526908),
 (4164, 0.007898642064658807),
 (71, 0.007458111932122567),
 (40, 0.00717549230135534),
 (391, 0.007097215625850389),
 (4127, 0.0068143792248614625),
 (2529, 0.006718650715440008),
 (144, 0.0067126822143698224),
 (1386, 0.0065222516071733105),
 (2927, 0.006040068555148989),
 (1847, 0.005909368174099682),
 

In [37]:
bclist = []
replist = []
reptroll = []
idlist = []

print(userrep['2047'])


for i in bc.items():
    #print("id = ",  )
    bclist.append(i[1])
    idlist.append(i[0])
    reptroll.append(userrep[str(i[0])] * 0.0000001)
    replist.append(userrep[str(i[0])])

for i in range(len(idlist)):
    print(idlist[i]," --> rep = ", replist[i], " | bc = ", bclist[i])
    

66
2054  --> rep =  33  | bc =  0.0
7  --> rep =  758  | bc =  8.35991198140731e-05
1034  --> rep =  281  | bc =  0.0
11  --> rep =  573  | bc =  0.00023445612668743508
2060  --> rep =  88  | bc =  0.0
13  --> rep =  440  | bc =  0.002643363499817939
2061  --> rep =  138  | bc =  0.0
1039  --> rep =  1444  | bc =  0.01737103744624187
15  --> rep =  1145  | bc =  0.0
3088  --> rep =  28  | bc =  0.0
18  --> rep =  750  | bc =  0.001194833852544133
14  --> rep =  149  | bc =  0.0
1044  --> rep =  148  | bc =  0.0
23  --> rep =  2081  | bc =  0.001681026090342679
24  --> rep =  286  | bc =  0.0
3097  --> rep =  149  | bc =  0.0
26  --> rep =  5220  | bc =  0.05307839567623448
27  --> rep =  394  | bc =  0.002044392523364486
4120  --> rep =  41  | bc =  0.0
29  --> rep =  190  | bc =  0.0
2074  --> rep =  36  | bc =  0.0
4127  --> rep =  480  | bc =  0.014092105270551526
2080  --> rep =  138  | bc =  0.0
31  --> rep =  250  | bc =  0.0
32  --> rep =  724  | bc =  0.0
35  --> rep =  353  | 

In [39]:
stats.ttest_ind(replist,bclist)

Ttest_indResult(statistic=7.630901915705073, pvalue=8.45763953399091e-14)